In [2]:
#!pip install simpletransformers

In [4]:
#!pip install torchvision

In [11]:
#import tensorflow as tf

In [1]:
import numpy as np
import pandas as pd
import warnings
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import json
import tensorflow_hub as hub
import tensorflow_text
import tldextract   # Accurately separates a URL's subdomain, domain, and public suffix
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import re
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from imblearn.over_sampling import SMOTE,ADASYN
from collections import Counter
import nltk 
import spacy
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from wordcloud import WordCloud, ImageColorGenerator,STOPWORDS
import gensim
from gensim.parsing.preprocessing import STOPWORDS as gensim_stopwords
import copy
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords,wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,LancasterStemmer
from sklearn.metrics import f1_score
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', 1000)

2022-06-19 20:14:00.955981: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-06-19 20:14:01.674480: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/software/ada/python/anaconda/2020.11/3.8/lib:/gpfs/software/ada/cuda/10.2.89/lib64:/gpfs/software/ada/cuda/10.2.89/lib/lib64
2022-06-19 20:14:01.674537: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package stopwords to
[nltk_data]     /gpfs/home/psc21zcu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Download

In [2]:
pd.set_option('display.max_colwidth', 200)
#pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)

In [3]:
foldr_pol_real = 'input_dataset//fakenewsnet//politifact//real//'
foldr_pol_fake = 'input_dataset//fakenewsnet//politifact//fake//'
foldr_gos_real = 'input_dataset//fakenewsnet//gossipcop//real//'
foldr_gos_fake = 'input_dataset//fakenewsnet//gossipcop//fake//'

##### Function to extract or convert JSON content to News Content format in dataframe

In [4]:
def json_to_newscontent(datapath):
    dictlist = []
    cols = ['url','text','title','authors','num_images','domain']
    folders = glob.glob(datapath+'/*')
    for index, subdir in enumerate(folders):
        path_file = glob.glob(subdir+'/*')
        #check whether file path is valid or not
        if len(path_file) == 1:
            file = open(path_file[0]).read()
            jsondata = json.loads(file)
            thedict = {'url':jsondata['url'],'title':jsondata['title'],'text':jsondata['text'],
                   'num_images':len(jsondata['images']),'authors':str(jsondata['authors'])}
            extrt_url = tldextract.extract(jsondata['url'])
            thedict['domain'] = extrt_url.domain
            dictlist.append(thedict)
    df=pd.DataFrame(dictlist,columns=cols)
    return df

##### Converting seperate dataframes for Politfact fake and real and Gossip fake and real

In [5]:
import time

# get the start time
st = time.time()

df_pol_real = json_to_newscontent(foldr_pol_real)
df_pol_fake = json_to_newscontent(foldr_pol_fake)
df_gos_real = json_to_newscontent(foldr_gos_real)
df_gos_fake = json_to_newscontent(foldr_gos_fake)

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 312.7435884475708 seconds


In [6]:
## Cleaned dataset
df_pol_real = df_pol_real.loc[df_pol_real["text"] != '' ]
df_pol_fake = df_pol_fake.loc[df_pol_fake["text"] != '' ]
df_gos_real = df_gos_real.loc[df_gos_real["text"] != '' ]
df_gos_fake = df_gos_fake.loc[df_gos_fake["text"] != '' ]

df_pol_real.reset_index(drop=True,inplace=True)
df_pol_fake.reset_index(drop=True,inplace=True)
df_gos_real.reset_index(drop=True,inplace=True)
df_gos_fake.reset_index(drop=True,inplace=True)

In [7]:
df_pol_real['Category'] = 'Politics' 
df_pol_fake['Category'] = 'Politics'
df_gos_real['Category'] = 'Gossips'
df_gos_fake['Category'] = 'Gossips'

##### Add response variable 'class' based on True=1 or Fake=0

In [8]:
df_pol_real['class'] = 0 #True
df_pol_fake['class'] = 1 #Fake
df_gos_real['class'] = 0
df_gos_fake['class'] = 1

##### Storing dataframe to csv file 

In [11]:
df_pol_real.to_csv('Updated//df_pol_real.csv', index=False)
df_pol_fake.to_csv('Updated//df_pol_fake.csv', index=False)
df_gos_real.to_csv('Updated//df_gos_real.csv', index=False)
df_gos_fake.to_csv('Updated//df_gos_fake.csv', index=False)

#### Creating dataframe from csv

In [13]:
# #creating a data frame
df_pol_real = pd.read_csv("Updated//df_pol_real.csv")
df_pol_fake = pd.read_csv("Updated//df_pol_fake.csv")
df_gos_real = pd.read_csv("Updated//df_gos_real.csv")
df_gos_fake = pd.read_csv("Updated//df_gos_fake.csv")

In [ ]:
# df_pol_real.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
# df_pol_fake.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
# df_gos_real.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
# df_gos_fake.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [14]:
df_list = [df_pol_real,df_pol_fake,df_gos_real,df_gos_fake]

In [15]:
#combine df's into one
df = pd.concat(df_list).reset_index(drop=True)
total_rows = df.shape[0]

In [16]:
df['class'].value_counts() / df.shape[0]*100

0    75.193989
1    24.806011
Name: class, dtype: float64

##### combining title and text to full text column

In [17]:
df['fulltext']=df['title']+'.'+df['text']
#df2=df2.drop(columns=['url', 'text','title','authors','num_images','domain','Category','url_filter'], axis=1)
df=df.dropna()
df.shape

(20362, 9)

### Saving Fulldataset

In [3]:
#df.to_csv('Updated//full_dataset.csv', index=False)

### Loading the Dataset

In [5]:
#creating a data frame
df = pd.read_csv("Updated//full_dataset.csv")

In [6]:
filterd_df=df[['fulltext','class']]

In [7]:
filterd_df.head(2)

,fulltext,class
0,"Rick Santorum: By amending constitution, we can balance the budget for good.Rick Santorum: By amending constitution, we can balance the budget for good\n\nBy RICK SANTORUM\n\nLast summer, we saw a...",0
1,"CNN's Don Lemon: Bill O'Reilly's Criticism Of Black Community ""Doesn't Go Far Enough"".Posted on July 27, 2013\n\nCNN's Don Lemon: Bill O'Reilly's Criticism Of Black Community ""Doesn't Go Far Enoug...",0


### Cleaning data

In [9]:
# Using the built in cleaner in gensim for basic text preprocessing
# filterd_df['cleantext'] = filterd_df['fulltext'].apply(lambda x: gensim.utils.simple_preprocess(x))
# filterd_df.head(1)

In [ ]:
### Transforming and creation of dataframe with Cleaned Text Column and class label
## Tranforming **cleanedtext** column from list of arrays to string 

In [6]:
# filterd_df['cleantext'] = filterd_df['cleantext'].apply(lambda x: x[0:-1])
# filterd_df['cleantext'] = [','.join(map(str, l)) for l in filterd_df['cleantext']]

In [8]:
# def clean_format(text):
#     clean_text=re.sub('\[[^]]*\]', ' ', str(text))
#     clean_text = re.sub('[^a-zA-Z]',' ',clean_text)  # replaces non-alphabets with spaces
#     clean_text=re.sub(r' {2,}',' ',clean_text)
#     return clean_text

In [9]:
# filterd_df['cleantext']=filterd_df['cleantext'].apply(clean_format)

In [10]:
# filterd_df=filterd_df[['cleantext','class']]

In [11]:
# filterd_df = filterd_df.sample(frac=1).reset_index(drop=True)

In [12]:
# filterd_df.reset_index(drop=True,inplace=True)

In [10]:
filterd_df.head(1)

,fulltext,class
0,"Rick Santorum: By amending constitution, we can balance the budget for good.Rick Santorum: By amending constitution, we can balance the budget for good\n\nBy RICK SANTORUM\n\nLast summer, we saw a great groundswell of national support to get our fiscal house in order by cutting and capping federal spending and passing a balanced budget amendment to the Constitution. However, the President refused to lead and instead pushed the tough decisions to a bureaucratic Washington committee to solve the problem, and though it was named the 'Supercommittee' it couldn't even agree on cutting Post-It Notes from the office supply closet. Now, because of the inaction, we are faced with drastic defense cuts at a time when reports show a nuclear Iran has emerged as a clear and present danger.I supported the 'Cut, Cap, and Balance' movement, but the reality is that what we really need is 'Balance, Balance, Grow' - because without a balanced budget amendment, and a path to economic prosperity, we can...",0


### BERT Embeddings

In [2]:
# https://www.tensorflow.org/text/tutorials/classify_text_with_bert
#https://androidkt.com/simple-text-classification-using-bert-in-tensorflow-keras-2-0/

In [8]:
texts=filterd_df['fulltext']

In [9]:
# bert preprocessor - https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# bert encoder - https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/2
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/2",trainable=True)

2022-06-19 20:14:28.150575: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/software/ada/python/anaconda/2020.11/3.8/lib:/gpfs/software/ada/cuda/10.2.89/lib64:/gpfs/software/ada/cuda/10.2.89/lib/lib64
2022-06-19 20:14:28.150780: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/software/ada/python/anaconda/2020.11/3.8/lib:/gpfs/software/ada/cuda/10.2.89/lib64:/gpfs/software/ada/cuda/10.2.89/lib/lib64
2022-06-19 20:14:28.150926: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH

In [10]:
# preprocessing dataset  - First Set
inputs = preprocessor(texts[0:10000])
# feeding it to model for vectorization
outputs = encoder(inputs)

In [11]:
len(texts)

20362

In [12]:
outputs.keys()

dict_keys(['sequence_output', 'encoder_outputs', 'pooled_output', 'default'])

In [13]:
pooled_output = outputs["pooled_output"]      # [batch_size, 512].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 512].

In [14]:
pooled_output.shape

TensorShape([10000, 512])

In [15]:
sequence_output.shape

TensorShape([10000, 128, 512])

In [16]:
# defining dataframe
bertf_df1=pd.DataFrame()

In [17]:
## Converting bert encoder sequence output to 1 dimension for ML Model training

for i in range(0,len(sequence_output)):
    b=sequence_output[i].numpy().sum(axis=0)
    bertf_df1=bertf_df1.append(pd.Series(b),ignore_index=True)
print('values added in dataframe')

values added in dataframe


In [18]:
bertf_df1.shape

(10000, 512)

In [19]:
# preprocessing dataset  - Second Set
inputs2 = preprocessor(texts[10000:21000])
# feeding it to model for vectorization
outputs2 = encoder(inputs2)

In [20]:
pooled_output2 = outputs2["pooled_output"]      # [batch_size, 512].
sequence_output2 = outputs2["sequence_output"]  # [batch_size, seq_length, 512].

In [21]:
bertf_df2=pd.DataFrame()

In [22]:
for i in range(0,len(sequence_output2)):
    b=sequence_output2[i].numpy().sum(axis=0)
    bertf_df2=bertf_df2.append(pd.Series(b),ignore_index=True)
print('values added in dataframe')

values added in dataframe


In [27]:
# merging both props 
bertVectors_fulldf=pd.concat([bertf_df1,bertf_df2])
bertVectors_fulldf.reset_index(drop=True,inplace=True)

In [28]:
# adding class labels
bertVectors_fulldf.insert(len(bertVectors_fulldf.columns),'class',filterd_df['class'])

In [29]:
bertVectors_fulldf['class'].value_counts()

0    15311
1     5051
Name: class, dtype: int64

In [30]:
bertVectors_fulldf.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,class
0,29.556551,17.010456,31.388668,-59.826038,22.785978,-40.389637,63.132030,84.202469,-16.992029,-23.151354,43.602962,-49.792343,19.467327,-48.555180,-24.660128,23.610239,68.322159,-24.381779,-0.276951,-0.942664,-23.207857,22.574673,-25.435923,-8.476034,24.444014,-9.494406,4.695382,41.938358,-6.322362,-27.365942,3.681536,-79.809784,44.775520,-38.007759,-14.645349,14.415798,9.520861,-65.712296,5.570489,68.280205,14.519122,-17.480473,46.555386,-24.409235,60.196117,51.608974,20.218319,111.037277,20.729141,6.164212,40.037346,20.915165,-49.550720,-8.859608,17.657154,-64.730644,4.080844,25.650654,11.049224,39.231117,1.717297,-38.349976,-51.245243,62.720028,13.244308,56.438663,-29.142439,17.777523,16.075716,32.840721,73.021126,-1.106223,43.724163,110.561844,5.694403,-3.568817,-6.633853,-20.060560,-20.269951,-90.533340,-27.684378,2.509365,0.751597,59.583221,-31.484814,-37.511806,-12.505559,-8.637518,-2.638591,57.80661,-27.497965,70.625862,5.287386,-10.089879,-24.389599,-21.589941,-29.910767,-16.954830,5.852044,67.251831,12.467425,26.722008,74.665611,-45.834213,-27.048599,46.073143,-10.609952,-4.713609,-37.950188,22.113008,22.008308,19.812160,-87.337875,-59.416115,-5.307872,37.954582,-58.814655,-40.410023,67.103203,-9.136406,-10.947894,-9.197604,-19.980762,12.875637,4.647591,46.636059,-30.788107,-38.222744,36.502457,7.888191,-53.950577,-38.763725,4.683434,-76.262459,-12.632508,33.230438,-33.218147,-38.273003,60.519566,-55.548923,21.099266,24.372196,19.415829,48.136097,54.441914,-29.664719,-9.153059,61.043640,-31.490051,-112.432663,-19.331478,46.725330,9.383484,14.873656,-1.091031,1.271348,3.287261,-2.009163,-30.760050,-30.436218,14.952808,18.419436,29.612751,59.370369,34.593708,-19.635996,50.557961,-36.320408,-65.584396,30.164946,98.545799,-26.293577,-4.078367,10.270546,23.894571,-6.337955,16.216356,39.860119,74.214920,-38.005817,47.595745,5.964079,-849.917603,34.075047,-11.063751,17.136820,-25.357260,-0.509272,13.924665,-38.921009,-22.454096,34.336700,59.178024,39.083691,39.124416,-1.147380,-39.004841,-22.438942,-42.472679,-8.890372,-53.831310,15.963998,5.907835,63.313263,50.1

### Saving Bertvectors in CSV

In [ ]:
bertVectors_fulldf.to_csv('Updated//bertVectors_fulldf.csv', index=False)

### Loading dataframe BertVectors

In [2]:
# #creating a data frame
bertVectors_fulldf = pd.read_csv("Updated//bertVectors_fulldf.csv")

### Splitting dataset into train and test

In [31]:
X=bertVectors_fulldf.drop('class',axis=1)
y=bertVectors_fulldf['class']

#### Using StratifiedKFold

In [32]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [33]:
for i, j in kfold.split(X, y):
    # select rows
    X_train, X_test = X.iloc[i], X.iloc[j]
    y_train, y_test = y.iloc[i], y.iloc[j]

In [34]:
X_test.shape

(4072, 512)

#### Helper Functions

In [35]:
import timeit
from timeit import default_timer as timer
from datetime import timedelta
 


def metrics(X_test,y_test,clf):
    predictions=clf.predict(X_test)
    #predictions=(clf.predict_proba(X_test)[:,1] >= 0.3).astype(bool)
    print("confusion_matrix :")
    print(confusion_matrix(y_test,predictions))
    print("Accuracy Score :")
    print(accuracy_score(y_test, predictions))
    print("Classification Report :")
    print(classification_report(y_test, predictions))
    print("F1 score :")
    print(f1_score(y_test, predictions))
    print("ROC AUC Score")
    y_pred_proba = clf.predict_proba(X_test)
    print(roc_auc_score(y_test, y_pred_proba[:,1]) )
    print("------------------------------")

    
def model_comparison_table(X_test,y_test,classifier_list):
    dict_clf={}
    for clf_name,clf in classifier_list:
        predictions=clf.predict(X_test)
        y_pred_proba = clf.predict_proba(X_test)
        accuracy=accuracy_score(y_test, predictions)
        precision=precision_score(y_test,predictions,average='macro').round(2)
        recall=recall_score(y_test,predictions)
        f1score=f1_score(y_test,predictions).round(2)
        ROC_AUC=roc_auc_score(y_test, y_pred_proba[:,1])
        dict_clf[clf_name]=[accuracy,precision,recall,f1score,ROC_AUC]
    df_models_scores = pd.DataFrame(dict_clf, index=['Accuracy', 'Precision', 'Recall', 'F1 Score','roc_auc_score'])
    return df_models_scores

## Random Forest

In [36]:
# Instantiate and fit a basic Random Forest model on top of the vectors
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf_model = rf.fit(X_train, y_train)

In [37]:
# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, y_pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, y_pred)))
print("Classification Report")
print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
print("Precision : {}".format(precision))
recall = recall_score(y_test, y_pred)
print("Recall : {}".format(recall))

Accuracy score : 0.81286836935167
Confusion matrix : 
 [[3008   54]
 [ 708  302]]
Classification Report
              precision    recall  f1-score   support

           0       0.81      0.98      0.89      3062
           1       0.85      0.30      0.44      1010

    accuracy                           0.81      4072
   macro avg       0.83      0.64      0.66      4072
weighted avg       0.82      0.81      0.78      4072

Precision : 0.848314606741573
Recall : 0.299009900990099


##### Hyperparameter Tuning

In [9]:
# Number of trees in random forest
n_estimators =[int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth =[int(x) for x in np.linspace(10, 110, num = 11)]
#Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {
    'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap
              }
pprint(random_grid)



{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}


In [10]:
rf_clf1=RandomForestClassifier(random_state=0)

In [11]:
rf_random = RandomizedSearchCV(estimator=rf_clf1, param_distributions=random_grid,
                              n_iter = 100, 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=False, max_depth=80, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=900; total time= 6.9min
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=900; total time= 2.6min
[CV] END bootstrap=True, max_depth=60, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time= 5.0min
[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time= 2.6min
[CV] END bootstrap=True, max_depth=60, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time= 2.5min
[CV] END bootstrap=True, max_depth=70, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=  46.8s
[CV] END bootstrap=True, max_depth=100, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600;

[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1000; total time= 2.7min
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=800; total time= 5.2min
[CV] END bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=1000; total time= 7.0min
[CV] END bootstrap=False, max_depth=70, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=500; total time= 3.4min
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=1000; total time= 4.2min
[CV] END bootstrap=False, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=  38.7s
[CV] END bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=900; total time= 5.8min
[CV] END bootstrap=False, max_depth=1

[CV] END bootstrap=False, max_depth=90, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=1000; total time= 6.4min
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1000; total time= 4.8min
[CV] END bootstrap=True, max_depth=110, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time= 1.4min
[CV] END bootstrap=False, max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=1000; total time= 4.5min
[CV] END bootstrap=False, max_depth=100, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time= 2.0min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=600; total time= 2.2min
[CV] END bootstrap=False, max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=100; total time=  34.6s
[CV] END bootstrap=False, max_depth

[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1000; total time= 2.7min
[CV] END bootstrap=True, max_depth=70, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=  45.1s
[CV] END bootstrap=True, max_depth=100, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time= 2.2min
[CV] END bootstrap=True, max_depth=60, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=300; total time= 1.4min
[CV] END bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=800; total time= 5.7min
[CV] END bootstrap=True, max_depth=70, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=900; total time= 3.8min
[CV] END bootstrap=True, max_depth=90, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=  27.5s
[CV] END bootstrap=True, max_depth=40, m

[CV] END bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  29.9s
[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=  42.3s
[CV] END bootstrap=False, max_depth=70, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=600; total time= 4.0min
[CV] END bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time= 1.4min
[CV] END bootstrap=False, max_depth=110, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=1000; total time= 7.3min
[CV] END bootstrap=False, max_depth=100, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time= 6.4min
[CV] END bootstrap=False, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=1000; total time= 4.2min
[CV] END bootstrap=False, max_depth=

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=0),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]},
                   random_state=42, return_train_score=True, verbose=2)

In [13]:
rf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 70,
 'bootstrap': False}

[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=500; total time= 3.0min
[CV] END bootstrap=True, max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  23.9s
[CV] END bootstrap=True, max_depth=100, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time= 2.4min
[CV] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=100; total time=  22.9s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=  16.9s
[CV] END bootstrap=True, max_depth=60, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time= 1.3min
[CV] END bootstrap=True, max_depth=100, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=800; total time= 3.0min
[CV] END bootstrap=False, max_depth=80, max_

#### Using with Best Params 

In [38]:
rf_clf_tuned=RandomForestClassifier(random_state=0,n_estimators=600,max_features='auto',max_depth=70,min_samples_split=2,min_samples_leaf=2,bootstrap=False)

rf_clf_tuned.fit(X_train, y_train)

pred1 = rf_clf_tuned.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, pred1)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred1)))

Accuracy score : 0.8197445972495089
Confusion matrix : 
 [[3014   48]
 [ 686  324]]


In [39]:
print("F1 score :")
print(f1_score(y_test, pred1))
print("Classification Report")
print(classification_report(y_test, pred1))

F1 score :
0.46888567293777134
Classification Report
              precision    recall  f1-score   support

           0       0.81      0.98      0.89      3062
           1       0.87      0.32      0.47      1010

    accuracy                           0.82      4072
   macro avg       0.84      0.65      0.68      4072
weighted avg       0.83      0.82      0.79      4072



### Support Vector Machine

In [17]:
svm_clf=SVC(random_state=0)

In [18]:
svm_clf.fit(X_train, y_train)

pred = svm_clf.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.8266208251473477
Confusion matrix : 
 [[2967   95]
 [ 611  399]]
Classification Report
              precision    recall  f1-score   support

           0       0.83      0.97      0.89      3062
           1       0.81      0.40      0.53      1010

    accuracy                           0.83      4072
   macro avg       0.82      0.68      0.71      4072
weighted avg       0.82      0.83      0.80      4072



#### Hyperparameter Tuning

In [21]:
# Create the random grid
random_grid = {
    'C': [0.1, 1, 10, 100, 1000],
               'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
               'kernel':  ['rbf', 'poly', 'sigmoid'],
              }
pprint(random_grid)


{'C': [0.1, 1, 10, 100, 1000],
 'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
 'kernel': ['rbf', 'poly', 'sigmoid']}


In [23]:

svm_clf1=SVC(random_state=0)

rf_random = RandomizedSearchCV(estimator=svm_clf1, param_distributions=random_grid,
                              n_iter = 50, 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] END ....................C=10, gamma=0.1, kernel=sigmoid; total time= 1.1min
[CV] END ...................C=0.1, gamma=0.1, kernel=sigmoid; total time= 1.1min
[CV] END .....................C=1000, gamma=0.01, kernel=rbf; total time= 5.3min
[CV] END ...............C=1000, gamma=0.0001, kernel=sigmoid; total time=  41.0s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 3.4min
[CV] END ..................C=1000, gamma=0.0001, kernel=poly; total time= 1.7min
[CV] END .........................C=1, gamma=0.1, kernel=rbf; total time= 5.9min
[CV] END .....................C=100, gamma=1, kernel=sigmoid; total time=  55.1s
[CV] END ...................C=100, gamma=0.1, kernel=sigmoid; total time=  44.5s
[CV] END ...............C=1000, gamma=0.0001, kernel=sigmoid; total time=  43.8s
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 3.4min
[CV] END ..................C=1000, gamma=0.0001

KeyboardInterrupt: 

In [ ]:
rf_random.best_params_

#### Using with Best Params 

In [26]:
svm_clf_tuned=SVC(random_state=0,probability=True)

svm_clf_tuned.fit(X_train, y_train)

pred1 = svm_clf_tuned.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, pred1)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred1)))

Accuracy score : 0.8266208251473477
Confusion matrix : 
 [[2967   95]
 [ 611  399]]


In [27]:
print("F1 score :")
print(f1_score(y_test, pred1))
print("Classification Report")
print(classification_report(y_test, pred1))

F1 score :
0.5305851063829787
Classification Report
              precision    recall  f1-score   support

           0       0.83      0.97      0.89      3062
           1       0.81      0.40      0.53      1010

    accuracy                           0.83      4072
   macro avg       0.82      0.68      0.71      4072
weighted avg       0.82      0.83      0.80      4072



## KNN

In [28]:
#Create KNN Object.
knn_clf = KNeighborsClassifier()

In [29]:
knn_clf.fit(X_train,y_train)

pred = knn_clf.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.81213163064833
Confusion matrix : 
 [[2794  268]
 [ 497  513]]
Classification Report
              precision    recall  f1-score   support

           0       0.85      0.91      0.88      3062
           1       0.66      0.51      0.57      1010

    accuracy                           0.81      4072
   macro avg       0.75      0.71      0.73      4072
weighted avg       0.80      0.81      0.80      4072



#### Hyperparameter Tuning

In [30]:
# Create the random grid
random_grid = {
    'leaf_size': list(range(1,50)),
               'n_neighbors': list(range(1,30)),
               'p':  [1,2],
              }
pprint(random_grid)

{'leaf_size': [1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
               47,
               48,
               49],
 'n_neighbors': [1,
                 2,
                 3,
                 

In [31]:

knn_clf1=KNeighborsClassifier()

rf_random = RandomizedSearchCV(estimator=knn_clf1, param_distributions=random_grid,
                              n_iter = 100, 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END ..................leaf_size=17, n_neighbors=19, p=2; total time=   4.9s
[CV] END ..................leaf_size=14, n_neighbors=29, p=2; total time=   3.4s
[CV] END ...................leaf_size=10, n_neighbors=6, p=2; total time=   3.6s
[CV] END ..................leaf_size=15, n_neighbors=15, p=2; total time=   3.1s
[CV] END ...................leaf_size=38, n_neighbors=8, p=2; total time=   3.4s
[CV] END ..................leaf_size=41, n_neighbors=27, p=2; total time=   3.0s
[CV] END ...................leaf_size=14, n_neighbors=6, p=2; total time=   3.3s
[CV] END ...................leaf_size=3, n_neighbors=13, p=2; total time=   3.0s
[CV] END ..................leaf_size=18, n_neighbors=16, p=2; total time=   3.0s
[CV] END ..................leaf_size=21, n_neighbors=24, p=1; total time= 2.1min
[CV] END ..................leaf_size=27, n_neighbors=16, p=1; total time= 2.3min


RandomizedSearchCV(cv=3, estimator=KNeighborsClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'leaf_size': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                      10, 11, 12, 13, 14, 15,
                                                      16, 17, 18, 19, 20, 21,
                                                      22, 23, 24, 25, 26, 27,
                                                      28, 29, 30, ...],
                                        'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8,
                                                        9, 10, 11, 12, 13, 14,
                                                        15, 16, 17, 18, 19, 20,
                                                        21, 22, 23, 24, 25, 26,
                                                        27, 28, 29],
                                        'p': [1, 2]},
                   random_state=42, return_train_score=True, verbose=2

In [33]:
rf_random.best_params_

{'p': 1, 'n_neighbors': 27, 'leaf_size': 13}

#### Using with Best Params 

In [34]:
knn_clf_tuned=KNeighborsClassifier(n_neighbors=27,leaf_size=13,p=1)
knn_clf_tuned.fit(X_train,y_train)

pred = knn_clf_tuned.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))
print("F1 score :")
print(f1_score(y_test, pred))

Accuracy score : 0.8180255402750491
Confusion matrix : 
 [[2908  154]
 [ 587  423]]
Classification Report
              precision    recall  f1-score   support

           0       0.83      0.95      0.89      3062
           1       0.73      0.42      0.53      1010

    accuracy                           0.82      4072
   macro avg       0.78      0.68      0.71      4072
weighted avg       0.81      0.82      0.80      4072

F1 score :
0.5330812854442344


### Xgboost

In [35]:
#Create KNN Object.
xg_clf=XGBClassifier(random_state=0)

xg_clf.fit(X_train,y_train)

pred = xg_clf.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))


Accuracy score : 0.8295677799607073
Confusion matrix : 
 [[2889  173]
 [ 521  489]]
Classification Report
              precision    recall  f1-score   support

           0       0.85      0.94      0.89      3062
           1       0.74      0.48      0.58      1010

    accuracy                           0.83      4072
   macro avg       0.79      0.71      0.74      4072
weighted avg       0.82      0.83      0.82      4072



#### HYperparameter Tuning

In [36]:
# Create the random grid
random_grid = {
    'min_child_weight': range(1,6,2),
    'gamma': list(range(1,30)),
    'max_depth':  range(3,15,2),
    'alpha': [1e-5, 1e-2, 0.1,1,10,50, 100],
    'subsample':np.arange(0.5, 1.0, 0.1),
    'scale_pos_weight':range(1,5,1),
    'colsample_bytree':np.arange(0.5, 1.0, 0.1),
    'n_estimators':[100, 250, 500],
    'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
    'colsample_bylevel': np.arange(0.5, 1.0, 0.1),
    }
pprint(random_grid)

{'alpha': [1e-05, 0.01, 0.1, 1, 10, 50, 100],
 'colsample_bylevel': array([0.5, 0.6, 0.7, 0.8, 0.9]),
 'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]),
 'gamma': [1,
           2,
           3,
           4,
           5,
           6,
           7,
           8,
           9,
           10,
           11,
           12,
           13,
           14,
           15,
           16,
           17,
           18,
           19,
           20,
           21,
           22,
           23,
           24,
           25,
           26,
           27,
           28,
           29],
 'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.4],
 'max_depth': range(3, 15, 2),
 'min_child_weight': range(1, 6, 2),
 'n_estimators': [100, 250, 500],
 'scale_pos_weight': range(1, 5),
 'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9])}


In [37]:

xg_clf1=XGBClassifier(random_state=0)


rf_random = RandomizedSearchCV(estimator=xg_clf1, param_distributions=random_grid,
                              n_iter = 100, 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END alpha=50, colsample_bylevel=0.6, colsample_bytree=0.8999999999999999, gamma=19, learning_rate=0.01, max_depth=5, min_child_weight=1, n_estimators=100, scale_pos_weight=2, subsample=0.7999999999999999; total time=  48.8s
[CV] END alpha=0.01, colsample_bylevel=0.5, colsample_bytree=0.7999999999999999, gamma=26, learning_rate=0.1, max_depth=7, min_child_weight=5, n_estimators=100, scale_pos_weight=3, subsample=0.7999999999999999; total time=  53.2s
[CV] END alpha=0.1, colsample_bylevel=0.7, colsample_bytree=0.6, gamma=27, learning_rate=0.2, max_depth=11, min_child_weight=3, n_estimators=250, scale_pos_weight=4, subsample=0.8999999999999999; total time= 3.6min
[CV] END alpha=50, colsample_bylevel=0.5, colsample_bytree=0.7999999999999999, gamma=22, learning_rate=0.4, max_depth=5, min_child_weight=5, n_estimators=500, scale_pos_weight=4, subsample=0.5; total time= 2.4min
[CV] END alpha=100, colsample_bylevel=0.8999999999

[CV] END alpha=100, colsample_bylevel=0.8999999999999999, colsample_bytree=0.6, gamma=15, learning_rate=0.4, max_depth=3, min_child_weight=1, n_estimators=500, scale_pos_weight=2, subsample=0.7; total time= 2.3min
[CV] END alpha=0.01, colsample_bylevel=0.7, colsample_bytree=0.8999999999999999, gamma=2, learning_rate=0.01, max_depth=13, min_child_weight=3, n_estimators=250, scale_pos_weight=3, subsample=0.5; total time= 4.0min
[CV] END alpha=0.1, colsample_bylevel=0.8999999999999999, colsample_bytree=0.7, gamma=22, learning_rate=0.3, max_depth=7, min_child_weight=5, n_estimators=500, scale_pos_weight=2, subsample=0.6; total time= 5.4min
[CV] END alpha=100, colsample_bylevel=0.7999999999999999, colsample_bytree=0.8999999999999999, gamma=3, learning_rate=0.1, max_depth=5, min_child_weight=3, n_estimators=500, scale_pos_weight=1, subsample=0.5; total time= 1.6min
[CV] END alpha=50, colsample_bylevel=0.8999999999999999, colsample_bytree=0.7999999999999999, gamma=28, learning_rate=0.4, max_d

[CV] END alpha=50, colsample_bylevel=0.7, colsample_bytree=0.6, gamma=1, learning_rate=0.3, max_depth=3, min_child_weight=1, n_estimators=500, scale_pos_weight=2, subsample=0.5; total time= 1.5min
[CV] END alpha=0.1, colsample_bylevel=0.7, colsample_bytree=0.7, gamma=14, learning_rate=0.4, max_depth=13, min_child_weight=1, n_estimators=100, scale_pos_weight=3, subsample=0.7999999999999999; total time= 1.8min
[CV] END alpha=1, colsample_bylevel=0.6, colsample_bytree=0.5, gamma=23, learning_rate=0.1, max_depth=11, min_child_weight=1, n_estimators=250, scale_pos_weight=4, subsample=0.7; total time= 2.5min
[CV] END alpha=1e-05, colsample_bylevel=0.6, colsample_bytree=0.8999999999999999, gamma=11, learning_rate=0.2, max_depth=3, min_child_weight=5, n_estimators=100, scale_pos_weight=2, subsample=0.8999999999999999; total time=  29.6s
[CV] END alpha=10, colsample_bylevel=0.8999999999999999, colsample_bytree=0.8999999999999999, gamma=20, learning_rate=0.1, max_depth=11, min_child_weight=3, n_

RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None, max_bin=None,...
                                        'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]),
                                        'gam

In [39]:
rf_random.best_params_

{'subsample': 0.7,
 'scale_pos_weight': 2,
 'n_estimators': 500,
 'min_child_weight': 1,
 'max_depth': 5,
 'learning_rate': 0.1,
 'gamma': 3,
 'colsample_bytree': 0.7999999999999999,
 'colsample_bylevel': 0.8999999999999999,
 'alpha': 1}

In [40]:
xg_clf_tuned=XGBClassifier(subsample=0.7,scale_pos_weight=2,n_estimators=500,min_child_weight=3,max_depth=5,learning_rate=0.1,
                     gamma=3,colsample_bytree=0.79,colsample_bylevel=0.89,alpha=1)
xg_clf_tuned.fit(X_train,y_train)

pred = xg_clf_tuned.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))
print("F1 score :")
print(f1_score(y_test, pred))

Accuracy score : 0.8320235756385069
Confusion matrix : 
 [[2834  228]
 [ 456  554]]
Classification Report
              precision    recall  f1-score   support

           0       0.86      0.93      0.89      3062
           1       0.71      0.55      0.62      1010

    accuracy                           0.83      4072
   macro avg       0.78      0.74      0.76      4072
weighted avg       0.82      0.83      0.82      4072

F1 score :
0.6183035714285714


### Naiva Bayes

In [42]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
nb_clf=GaussianNB()
#p = Pipeline([('Normalizing',MinMaxScaler()),('MultinomialNB',MultinomialNB())])
nb_clf.fit(X_train,y_train) 

pred = nb_clf.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.6863948919449901
Confusion matrix : 
 [[2102  960]
 [ 317  693]]
Classification Report
              precision    recall  f1-score   support

           0       0.87      0.69      0.77      3062
           1       0.42      0.69      0.52      1010

    accuracy                           0.69      4072
   macro avg       0.64      0.69      0.64      4072
weighted avg       0.76      0.69      0.71      4072



#### HYperparameter Tuning

In [43]:
# Create the random grid
random_grid = {
    'var_smoothing': [0.00001, 0.0001, 0.001, 0.1, 1, 10, 100,1000]
#    # 'class_prior':[True,False],
#     'fit_prior':[True,False]
              }
pprint(random_grid)

{'var_smoothing': [1e-05, 0.0001, 0.001, 0.1, 1, 10, 100, 1000]}


In [44]:

nb_clf1=GaussianNB()


rf_random = RandomizedSearchCV(estimator=nb_clf1, param_distributions=random_grid,
                              n_iter = 100, 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(X_train,y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


RandomizedSearchCV(cv=3, estimator=GaussianNB(), n_iter=100, n_jobs=-1,
                   param_distributions={'var_smoothing': [1e-05, 0.0001, 0.001,
                                                          0.1, 1, 10, 100,
                                                          1000]},
                   random_state=42, return_train_score=True, verbose=2)

In [71]:
nb_clf.get_params()

{'priors': None, 'var_smoothing': 1e-09}

In [45]:
rf_random.best_params_

{'var_smoothing': 1}

In [50]:
nb_clf_tuned=GaussianNB(var_smoothing=0.5)

nb_clf_tuned.fit(X_train,y_train)

pred = nb_clf_tuned.predict(X_test )
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))
print("F1 score :")
print(f1_score(y_test, pred))

Accuracy score : 0.7721021611001965
Confusion matrix : 
 [[3039   23]
 [ 905  105]]
Classification Report
              precision    recall  f1-score   support

           0       0.77      0.99      0.87      3062
           1       0.82      0.10      0.18      1010

    accuracy                           0.77      4072
   macro avg       0.80      0.55      0.53      4072
weighted avg       0.78      0.77      0.70      4072

F1 score :
0.18453427065026362


### Logistic Regression

In [52]:
from sklearn.linear_model import LogisticRegression

In [53]:
logticreg_clf=LogisticRegression()

logticreg_clf.fit(X_train,y_train)

pred = logticreg_clf.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.8172888015717092
Confusion matrix : 
 [[2846  216]
 [ 528  482]]
Classification Report
              precision    recall  f1-score   support

           0       0.84      0.93      0.88      3062
           1       0.69      0.48      0.56      1010

    accuracy                           0.82      4072
   macro avg       0.77      0.70      0.72      4072
weighted avg       0.81      0.82      0.81      4072



#### HYperparameter Tuning

In [54]:
# Create the random grid
random_grid = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
     'C':[100, 10, 1.0, 0.1, 0.01],
    'penalty':['l1','l2','elasticnet']
              }
pprint(random_grid)

{'C': [100, 10, 1.0, 0.1, 0.01],
 'penalty': ['l1', 'l2', 'elasticnet'],
 'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}


In [55]:

logticreg_clf_tuned=LogisticRegression()


rf_random = RandomizedSearchCV(estimator=logticreg_clf_tuned, param_distributions=random_grid,
                              n_iter = 100, 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)


# Fit the random search model
rf_random.fit(X_train,y_train)

Fitting 3 folds for each of 75 candidates, totalling 225 fits


RandomizedSearchCV(cv=3, estimator=LogisticRegression(), n_iter=100, n_jobs=-1,
                   param_distributions={'C': [100, 10, 1.0, 0.1, 0.01],
                                        'penalty': ['l1', 'l2', 'elasticnet'],
                                        'solver': ['newton-cg', 'lbfgs',
                                                   'liblinear', 'sag',
                                                   'saga']},
                   random_state=42, return_train_score=True, verbose=2)

In [56]:
rf_random.best_params_

{'solver': 'liblinear', 'penalty': 'l1', 'C': 0.01}

### Using Best Params

In [61]:
logticreg_clf_tuned=LogisticRegression(solver='liblinear',penalty='l1',C=0.01)

logticreg_clf_tuned.fit(X_train,y_train)

pred = logticreg_clf_tuned.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))
print("F1 score :")
print(f1_score(y_test, pred))

Accuracy score : 0.8175343811394892
Confusion matrix : 
 [[2855  207]
 [ 536  474]]
Classification Report
              precision    recall  f1-score   support

           0       0.84      0.93      0.88      3062
           1       0.70      0.47      0.56      1010

    accuracy                           0.82      4072
   macro avg       0.77      0.70      0.72      4072
weighted avg       0.81      0.82      0.80      4072

F1 score :
0.5606150206978119


### Consolidating all classifiers accuracy scores

In [62]:
classifiers = [('Logistic Regression',logticreg_clf_tuned),('Naive Bayes',nb_clf_tuned),('Xgboost',xg_clf_tuned),
              ('KNN',knn_clf_tuned),('Random Forest',rf_clf_tuned),("SVM",svm_clf_tuned)]

In [63]:
df_model_scores=model_comparison_table(X_test,y_test,classifiers)
df_model_scores.head(20).T.sort_values(by='Accuracy', ascending=False)

,Accuracy,Precision,Recall,F1 Score,roc_auc_score
Xgboost,0.832024,0.78,0.548515,0.62,0.839054
SVM,0.826621,0.82,0.395050,0.53,0.825470
Random Forest,0.824165,0.84,0.344554,0.49,0.822716
KNN,0.818026,0.78,0.418812,0.53,0.815061
Logistic Regression,0.817534,0.77,0.469307,0.56,0.816043
Naive Bayes,0.772102,0.80,0.103960,0.18,0.740386
